In [20]:
import numpy as np
# Реализуем класс узла

class Node:

    def __init__(self, index, t, true_branch, false_branch, level):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
        self.level = level # уровень узла

# И класс терминального узла (листа)
class Leaf:

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred

    def predict(self):
        return self.labels.mean()


# Разбиение датасета в узле

def split(data, labels, index, t):
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
    return true_data, false_data, true_labels, false_labels

def rss(labels):
    return np.sum((labels - labels.mean())**2)

# Расчет качества
def quality(left_labels, right_labels):
    return rss(left_labels) + rss(right_labels)


# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    best_rss = rss(labels)
    best_t = None
    best_index = None

    n_features = data.shape[1]

    for index in range(n_features):
        t_values = [row[index] for row in data]
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            ##  пропускаем разбиения, в которых в узле остается менее 5 объектов
            # if len(true_data) < min_leaf or len(false_data) < min_leaf:
            #    continue

            current_rss = quality(true_labels, false_labels)

            #  выбираем порог, на котором получается максимальный прирост качества
            if current_rss < best_rss:
                best_rss, best_t, best_index = current_rss, t, index

    return best_rss, best_t, best_index

# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, level = 0, depth = 0):

    RSS, t, index = find_best_split(data, labels)
    #  Базовый случай - прекращаем рекурсию, когда ошибка равна 0
    if rss(labels) == 0:
        return Leaf(data, labels)
    
    # Прекращаем рекурсию когда достигли заданной глубины дерева
    if depth != 0 and level == depth:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    level += 1
    true_branch = build_tree(true_data, true_labels, level, depth)
    false_branch = build_tree(false_data, false_labels, level, depth)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch, level)

# Построим дерево по обучающей выборке
X_train = np.array([[1],[2],[3],[4],[5]])
y_train = np.array([1,2,9,10,10])
my_tree = build_tree(X_train, y_train, depth=1)

depth=1, level=0
depth=1, level=1
depth=1, level=1


C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: Mean of empty slice.


In [21]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 0
Порог 2
--> True:
  Прогноз: 1.5
--> False:
  Прогноз: 9.666666666666666


In [22]:
# Проход объекта по дереву для его классификации
def pred_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return pred_object(obj, node.true_branch)
    else:
        return pred_object(obj, node.false_branch)

In [23]:
# Предсказание деревом для всего датасета

def predict(data, tree):
    
    y_pred = []
    for obj in data:
        prediction = pred_object(obj, tree)
        y_pred.append(prediction)
    return y_pred

In [24]:
# Получим ответы для обучающей выборки 
train_answers = predict(X_train, my_tree)
train_answers

[1.5, 1.5, 9.666666666666666, 9.666666666666666, 9.666666666666666]

In [25]:
X_test = np.array([[15],[0]])

In [26]:
# И получим ответы для тестовой выборки
answers = predict(X_test, my_tree)
answers

[9.666666666666666, 1.5]